In [1]:
%load_ext autoreload
%autoreload 2
import penzai
from penzai import pz
pz.ts.register_as_default()
pz.ts.register_autovisualize_magic()
pz.enable_interactive_context()

In [10]:
from micrlhf.llama import LlamaTransformer
llama = LlamaTransformer.from_pretrained("models/gemma-2b-it-q4_k.gguf", device_map="tpu:0",
                                         from_type="gemma",
                                        #  load_eager=True
                                         )

In [11]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("NousResearch/gemma-2b-it-tokenizer")
tokenizer.padding_side = "right"
format_prompt = """<start_of_turn>user\n
{}\n
<start_of_turn>model\n
{}"""

In [16]:
from micrlhf.sampling import sample
sample(llama, tokenizer, format_prompt.format("Hi!", ""),
       batch_size=4, do_sample=True)[0]

LoweringException: Exception while lowering eqn:
  a:u8[2,4,1,256] = and b 63
With context:
  LoweringRuleContext(lowering_context=LoweringContext(ir_context=<jaxlib.mlir._mlir_libs._site_initialize.<locals>.Context object at 0x7facb476a510>, grid_rank=2, mapped_dims=(), user_grid_indices=(<jaxlib.mlir._mlir_libs._mlir.ir.BlockArgument object at 0x7facd43268b0>, <jaxlib.mlir._mlir_libs._mlir.ir.BlockArgument object at 0x7facd4325c70>), block_shapes=[(8, 12, 256), (8, 128, 256), (8, 1, 256), (8, 1, 256), (256, 2048), (256, 256), None], name_stack=NameStack(stack=()), mesh_context=None, traceback_caches=TracebackCaches(traceback_cache={<jaxlib.xla_extension.Traceback object at 0x5577d17673e0>: loc(callsite("matmul_4bit_kernel"("/home/neverix/micrlhf-progress/micrlhf/quantizers.py":128:21) at callsite("matmul_fast.<locals>.kernel_call"("/home/neverix/micrlhf-progress/micrlhf/quantizers.py":208:18) at callsite("matmul_fast"("/home/neverix/micrlhf-progress/micrlhf/quantizers.py":232:13) at 